<a href="https://colab.research.google.com/github/SpaceSapiens/Study/blob/IITM/L6_5_Networks_Kumu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
os.listdir()

['.config', 'sample_data']

In [ ]:
# Download the data
!rm *.tsv.gz
!wget -q https://datasets.imdbws.com/name.basics.tsv.gz
!wget -q https://datasets.imdbws.com/title.principals.tsv.gz
!wget -q https://datasets.imdbws.com/title.basics.tsv.gz
!wget -q https://datasets.imdbws.com/title.akas.tsv.gz
!ls -la

rm: cannot remove '*.tsv.gz': No such file or directory
total 1117332
drwxr-xr-x 1 root root      4096 Mar 23 11:07 .
drwxr-xr-x 1 root root      4096 Mar 23 10:56 ..
drwxr-xr-x 4 root root      4096 Mar 21 22:53 .config
-rw-r--r-- 1 root root 242796056 Mar 22 13:16 name.basics.tsv.gz
drwxr-xr-x 1 root root      4096 Mar 21 22:54 sample_data
-rw-r--r-- 1 root root 299886705 Mar 22 13:16 title.akas.tsv.gz
-rw-r--r-- 1 root root 169658860 Mar 22 13:16 title.basics.tsv.gz
-rw-r--r-- 1 root root 431770513 Mar 22 13:16 title.principals.tsv.gz


In [ ]:
# Load the titles
title = pd.read_csv('title.basics.tsv.gz', sep='\t', low_memory=True).set_index('tconst')[['titleType', 'primaryTitle', 'startYear']]


<ipython-input-4-27b40326917d>:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  title = pd.read_csv('title.basics.tsv.gz', sep='\t', low_memory=True).set_index('tconst')[['titleType', 'primaryTitle', 'startYear']]


,titleType,primaryTitle,startYear
tconst,,,
tt0000001,short,Carmencita,1894
tt0000002,short,Le clown et ses chiens,1892
tt0000003,short,Pauvre Pierrot,1892
tt0000004,short,Un bon bock,1892
tt0000005,short,Blacksmith Scene,1893


In [ ]:
title.head()

In [ ]:
title.dtypes

In [ ]:
title=title[title['startYear']>='1990']

In [ ]:
# Load the cast of each film
cast =pd.read_csv('title.principals.tsv.gz', sep='\t', low_memory=True)[['tconst', 'nconst', 'category']]
# Only consider actors, not directors, composers, etc. Shrinks data to about 40%
cast = cast[cast.category.isin({'actor', 'actress'})]
cast.head()

NameError: ignored

In [ ]:
# Only consider movies, not TV series, etc. Shrinks data to ~5%
movies = title[title['titleType'] == 'movie']
cast = cast[cast['tconst'].isin(movies.index)]
# This is what the network looks like
cast.head()

In [ ]:
# Restrict data to just a single region (e.g. IN, US, etc)
# This loads the region for each title
region = pd.read_csv('title.akas.tsv.gz', sep='\t', low_memory=True).set_index('titleId')['region']
region.value_counts().head(10)

In [ ]:
region.head()

In [ ]:
# Load the name data along with birth year
name = pd.read_csv('name.basics.tsv.gz', sep='\t', na_values='\\N', dtype={'birthYear': float}).set_index('nconst')[['primaryName', 'birthYear']]

In [ ]:
name.head()

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
def get_pairs(lang=None, min_acted=1, min_pairings=1):
    '''
    Returns an adjacency matrix and actor mapping of actor pairs where:
    - Each actor has acted in at least min_acted films
    - The two actors have acted together in at least min_pairings films
    - And (optionally), belong to a region `lang` (IN, UN, etc)
    '''
    graph = cast
    if lang is not None:
        graph = graph[graph['tconst'].isin(region[region == lang].index)]
    name_freq = graph['nconst'].value_counts()
    top_names = name_freq[name_freq >= min_acted]
    top_actors = graph[graph['nconst'].isin(top_names.index)]

    p = top_actors.copy()
    p['title'] = p['tconst'].astype('category')
    p['name'] = p['nconst'].astype('category')

    row = p['title'].cat.codes.values
    col = p['name'].cat.codes.values
    data = np.ones(len(p), dtype='int')

    matrix = csr_matrix((data, (row, col)))
    square = matrix.T * matrix
    square.setdiag(0)
    square = square.tocoo()

    pairs = pd.DataFrame({
        'row': square.row,
        'col': square.col,
        'n': square.data
    })
    pairs = pairs[pairs.n >= min_pairings].reset_index(drop=True)
    return pairs, name.reindex(p['name'].cat.categories)

def lookup(pairs, cat):
    pairs = pd.concat([
        pairs,
        cat.iloc[pairs.row].reset_index(drop=True),
        cat.iloc[pairs.col].reset_index(drop=True),
    ], axis=1)
    pairs = pairs.drop(columns=['row', 'col'])
    pairs.columns = ['count', 'name1', 'year1', 'name2', 'year2']
    return pairs.sort_values('count', ascending=False)

In [ ]:
pairs, cat = get_pairs(lang='IN', min_acted=1, min_pairings=1)

In [ ]:
cat

In [ ]:
pairs

In [ ]:
ForKumu = lookup(pairs, cat)
ForKumu

In [ ]:
ForKumu = ForKumu[['name1', 'name2', 'count']]
ForKumu = ForKumu.rename(columns={'name1':'From',
                                  'name2':'To',
                                  'count':'Strength'})
ForKumu

In [ ]:
ForKumu.to_excel("pairs.xlsx", index = False)